In [1]:
from LSTMClass import *
# from GRUClass import *
from loader import *
import pdb
import pickle

In [2]:
path='~/'
name='hani_lstm585'
nameckp='hani_lstm'

In [3]:
conf,[w2idx,idx2w],[bValidLoss,train_loss]=pickle.load(open(path+name+'.pkl','rb'))

In [4]:
model = LSTM(embedding_size = conf['embedding_size'], 
             num_classes=conf['voc_size'], 
             lst_layers=conf['lst_layers'],
             ckpt_path=path)#path+nameckp)



<log> Building Graph...</log>


### Generate Sentences

In [5]:
gSent = model.GenerateSentence(w2idx,idx2w,5)

RESTORED
INFO:tensorflow:Restoring parameters from /home/hani/ModelOut/lstmfromscratch/lstmoldnews/hani_lstm585-31


In [6]:
for s in gSent:
    print(s,"\n")


SENTENCE_START it’s going to take us even more benefits , it could well seem to be cooling in later . SENTENCE_END 

SENTENCE_START bank of england budget eventually set policy on raise interest rates on nov. laid with fed hike planning more long-term “a home here and said it would keep purchases ratio as it receives lives near the charge , said the fed , in tells remained aim to achieve any special owner of collateral . SENTENCE_END 

SENTENCE_START while the figure has reached 2004 in recent days , it has cancelled changing conditions for growth of 0.6 % for the time being , both loan and confidence . SENTENCE_END 

SENTENCE_START 2008 forecasts yields would be the tons , slumped to 1 % on a sustained basis , he live doing so would it.the it , ” mr. symposium guidelines at work on carrying out the finish of level on financial p3 for continued this year . SENTENCE_END 

SENTENCE_START the bank’s policymakers are occur over china’s efforts to independence up the following help talks in

### Get Embeddings

In [7]:
embeddings=[]

sess = tf.Session() 
sess.run(tf.global_variables_initializer())
emb1=sess.run(model.embs)
# restore session
ckpt = tf.train.get_checkpoint_state(model.ckpt_path)
print(ckpt)
saver = tf.train.Saver()
saver.restore(sess, ckpt.model_checkpoint_path)
embeddings=sess.run(model.embs)
sess.close()

model_checkpoint_path: "/home/hani/ModelOut/lstmfromscratch/lstmoldnews/hani_lstm585-31"
all_model_checkpoint_paths: "/home/hani/ModelOut/lstmfromscratch/lstmoldnews/hani_lstm585-29"
all_model_checkpoint_paths: "/home/hani/ModelOut/lstmfromscratch/lstmoldnews/hani_lstm585-30"
all_model_checkpoint_paths: "/home/hani/ModelOut/lstmfromscratch/lstmoldnews/hani_lstm585-31"

INFO:tensorflow:Restoring parameters from /home/hani/ModelOut/lstmfromscratch/lstmoldnews/hani_lstm585-31


In [8]:
distanceArray=np.inf*np.ones([embeddings.shape[0],embeddings.shape[0]])
counter=0
while(counter<distanceArray.shape[0]):
    index=counter+1
    while(index<distanceArray.shape[1]):
        d = np.sum((embeddings[counter]-embeddings[index])**2)
        distanceArray[counter,index]=d
        distanceArray[index,counter]=d
        index+=1
    counter+=1
    
pickle.dump(distanceArray,open('~/distoldnews.pkl','wb'))    

### Build graph from distance

In [45]:

# f=open('/home/hani/ModelOut/lstmfromscratch/edges.csv','w')
# cutVal = np.percentile(np.reshape(distanceArray,-1),25)
# for i in range(distanceArray.shape[0]):
#     j=i+1
#     while j<distanceArray.shape[1]:
#         if distanceArray[i,j]>cutVal:
#             f.write(str(i)+","+str(j)+'\n')
#         j+=1
# f.close()
        


### Build Words similarity

In [9]:

# idx2w[0]='comma'
# distanceArray=pickle.load(open('/home/hani/ModelOut/lstmfromscratch/dist.pkl','rb'))
idx2w[w2idx[',']]='comma'
# distanceArray=pickle.load(open('/home/hani/ModelOut/lstmfromscratch/distoldnews.pkl','rb'))


delIndex=w2idx['UNKNOWN_TOKEN'],w2idx['SENTENCE_START'],w2idx['SENTENCE_END']
for t in delIndex:
    distanceArray[:,t]=np.inf
    
f=open('/home/hani/ModelOut/lstmfromscratch/lstmoldnews/wordsSimilarity.csv','w')

for i in range(distanceArray.shape[0]):
    sorted_index = np.argsort(distanceArray[i])[0:10]
    str_line=""
    str_line+=idx2w[i]
    for index in sorted_index:
        str_line+=','+idx2w[index]
    str_line+='\n'
    f.write(str_line)

f.close()            